In [6]:
"""
Program Description:
This program breaks down the data recorded on a QSense-Analyzer into independent, unified data sets, for each
Sensor position. It processes QCM-D data obtained with Q-Tools Software (Version 2.8.5) on QSense-Analyzer with open modules.

Author:
- Heike Böhm, GlycoScience group, Department of Cellular Biophysics, MPI for Medical Research (MPImF-CBP-GS)
- With strong support from Martin Schröter in the same department.

Input Data Sources:
- Rawdata as tab spaced CSV file exported from Q-Tools using "," as decimal points
    expects data to be saved with Data_ID_rawdata.txt
- Corresponding timeline copied and pasted from the notes window of Q-Tools into a plain text file.
    expects data to be saved with Data_ID_timeline.txt
    Format: h:mm:ss S# description. Timepoints should correspond to solution changes within a 4-minute timeframe.

Output Generated:
- A CSV file for each sensor file containing the following data: 
    - adj_qcmd_data: 
        Adjusted QCM-D dataset with time in seconds, normalized Δf/n, and normalized D for all overtones. 
          Normalization is achieved by subtracting the averaged value of the first 100 data points 
          for each measured frequency and dissipation overtone.
          All normalized frequency values have been divided by the overtone number to represent Δf/n.
    - adj_timeline
        timeline given in the text file for the respective sensor including the origianlly recorded time in s,
        the adjusted time as determined by a min or max within the 4min time window of the original time
        indicating the exchange of liquids in open modules and the information given on the solution change 
      
Comments on Coding:
- For all variables lower_case_with_underscores are used. 
- CapWords are used for class names 
- UPPER_CASE_WITH_UNDERSCORES are used for constants.
"""

import os
import ipywidgets as widgets
import numpy as np
import h5py
import pandas as pd
import re


In [7]:
'''
Enter the path to the folder in which the QCMD flow rawdata is saved
'''

input_widget_width = '1000px'
description_width = '300px'

path_widget = widgets.Text(
    value="/Users/hboehm/Seafile/LEAF/QCMD/QCMD_data/Inbox",
    description="Path to QCMD open module rawdata:"
)

path = None

path_widget.layout.width = input_widget_width
path_widget.style.description_width = description_width

def save_path(sender):
    global path 
    path = path_widget.value

save_button = widgets.Button(description="Save")
save_button.on_click(save_path)

widgets.VBox([path_widget, save_button])


In [8]:
"""
Input:
- Reads in the rawdata txt File with time in seconds as the index.

Output:
- persistentID: represents the uniqueID of the dataset - extracted from the name of the rawdata title 
- sensors: number of sensors for which data is available
- adj_qcmd_data as long format pandas dataframe
    - frequency and Dissipation values are normalized to start at zero by subtracting the averaged value of the first 100 data points.
    - frequency values are divided by the overtone number to represent Δf/n.

Note on Coding:
- The rawdata is expected to be exported with column titles in the format"Time_S" and "Mn_S [unit]"
    where M represents f for frequency or D for dissipation; n is the overtone number and S the sensor No.
"""

def process_rawdata(file_path):
    qcmd_data_wide = pd.read_csv(file_path, sep="\t", decimal=",", encoding='latin-1')
    rawdata = qcmd_data_wide
    rawdata_array = rawdata.to_numpy()  # to save unmodified rawdata in hdf5 file later
    rawdata_column_names = rawdata.columns

    persistentID = os.path.basename(file_path)
    persistentID = persistentID.replace("Data_", "").replace("_rawdata.txt", "")

    sensors = {int(item.split("_")[1].split(" ")[0]) for item in qcmd_data_wide if item[0] == "f"}
    overtones = {int(item.split("_")[0][1:]) for item in qcmd_data_wide if item[0] == "f"}

    print("Starting to process data with the persistent ID:", persistentID)
    print("In this dataset, the following overtones were measured:")
    print(overtones)
    print("on the sensor positions")
    print(sensors)

    norm_qcmd_data_wide = qcmd_data_wide

    # the data contains a Time column for each of the sensors. 
    # Thus we must ensure that none of the columns labeled "Time_n [s]" - with n being the overtone number - is "normalized
    for col in qcmd_data_wide.columns: 
        if not re.match(r'Time_\d+\s+\[s\]', col): 
            norm_qcmd_data_wide[col] = qcmd_data_wide[col]-qcmd_data_wide[col].iloc[0:100].mean()
            
    # Next step: Create adj_qcmd_data in long format
    adj_qcmd_data = pd.DataFrame(columns=["Time_s", "Overtone", "Sensor", "Deltaf_div_n_Hz", "Dissipation_ppm"])

    for sensor in sensors:
        for overtone in overtones:
            columns_selected = [
                "Time_" + str(sensor) + " [s]",
                "f" + str(overtone) + "_" + str(sensor) + " [Hz]",
                "D" + str(overtone) + "_" + str(sensor) + " [ppm]"
            ]

            data_temp = qcmd_data_wide.loc[:, columns_selected]
            data_temp.columns = ["Time_s", "Deltaf_div_n_Hz", "Dissipation_ppm"]
            data_temp["Overtone"] = float(overtone)
            data_temp["Sensor"] = float(sensor)

            adj_qcmd_data = pd.concat([adj_qcmd_data, data_temp], ignore_index=True)

    adj_qcmd_data["Deltaf_div_n_Hz"] = adj_qcmd_data["Deltaf_div_n_Hz"] / adj_qcmd_data["Overtone"]
    adj_qcmd_data["Deltaf_div_n_Hz"] = adj_qcmd_data["Deltaf_div_n_Hz"].astype(float)
    adj_qcmd_data["Time_s"] = adj_qcmd_data["Time_s"].astype(float)
    adj_qcmd_data["Dissipation_ppm"] = adj_qcmd_data["Dissipation_ppm"].astype(float)

    adj_qcmd_data = adj_qcmd_data.dropna()

    #print("Long-format data:")
    #display(adj_qcmd_data)
    print("Transformed to normalized long format")
    
    return persistentID, sensors, adj_qcmd_data


In [9]:
'''
Input:
- path
- persistenID
- Reads in the timeline txt based on our naming conventions 
    expects title of timeline to be path+"Data_"+persistentID+"_timeline.txt"
- adj_qcmd_data

Output:
- timeline_data_pd: adjusted time points at which liquid has been exchanged in a pandas data frame 
    column names: ['Time', 'Sensor', 'Information']
- extreme_values_times
  - Based on maximum and minimum values in a 4-minute timeframe of the original timepoints given in the timeline.
  - Corresponds to max_value for SUV-containing solutions.
  - Otherwise, calculates the difference between the extrema and the mean value over the timeframe, 
    returning the value with the larger difference.
- corr_timeline
  - timeline with corrected values based on extreme_values_times

Note on Coding:
- Remember to change the timeframe also in the beginning of the program if you adjust it here.
'''



# Function to find highest and lowest values within a 4-minute timeframe
def find_extreme_values(data, timepoints, window_size=240):
    extreme_values = []

    for row in timepoints:
        time, description = row['Time'], row['Information']
        #print("Time:", time, "Description:", description)

        start_time = float(time) - (window_size / 2)
        end_time = float(time) + (window_size / 2)

        # Filter the data within the specified timeframe
        data_within_timeframe = data[(data['Time_s'] >= start_time) & (data['Time_s'] <= end_time)]

        if not data_within_timeframe.empty:
            # Find the highest and lowest values within the filtered data
            max_value_index = data_within_timeframe['Deltaf_div_n_Hz'].idxmax()
            min_value_index = data_within_timeframe['Deltaf_div_n_Hz'].idxmin()

            # Convert indices to corresponding time values
            max_value_time = data_within_timeframe['Time_s'].loc[max_value_index]
            min_value_time = data_within_timeframe['Time_s'].loc[min_value_index]

            # Calculate mean and standard deviation
            mean = np.mean(data_within_timeframe['Deltaf_div_n_Hz'])

            # Calculate Z-scores for maximum and minimum values
            max_value = data_within_timeframe['Deltaf_div_n_Hz'].loc[max_value_index]
            min_value = data_within_timeframe['Deltaf_div_n_Hz'].loc[min_value_index]
            Diff_max = max_value -  mean
            Diff_min = mean - min_value

            if Diff_max > Diff_min:
                extreme_values.append(max_value_time)
                #print("Max Value:", max_value_time)
            else:
                extreme_values.append(min_value_time)
                #print("Min Value:", min_value_time)

    return extreme_values




def read_timeline_data(path, persistentID, adj_qcmd_data):
    file_timeline = os.path.join(path, f"Data_{persistentID}_timeline.txt")

    try:
        with open(file_timeline, 'r', encoding='utf-8') as file:
            timeline = file.read()
            print("Timeline file read:")
            print(f"Data_{persistentID}_timeline.txt")
    except FileNotFoundError:
        print(f"The file '{file_timeline}' was not found. Check if it was saved with the correct name")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

    # Initialize lists to store timeline data
    timelines_times = []
    timelines_sensors = []
    timelines_speeds = []
    timelines_infos = []

    timelines = timeline.split("\n")
    for line in timelines:
        line = line.strip()  # Remove leading and trailing whitespace

        # Check if the line is not empty
        if line:
            parts = line.split(" ", 2)
            time_str = parts[0]
            timelines_info = parts[2]

            timelines_sensor = int(parts[1].replace("S", ""))  # Remove 'S' prefix

            # Split the time string into hours, minutes, and seconds
            time_parts = time_str.split(":")

            # If there's no hours part, assume it's zero
            if len(time_parts) == 2:
                hours, minutes, seconds = 0, int(time_parts[0]), int(time_parts[1])
            elif len(time_parts) == 3:
                hours, minutes, seconds = map(int, time_parts)
            else:
                raise ValueError(f"Invalid time format: {time_str}")

            # Convert the time to seconds
            timelines_time = hours * 3600 + minutes * 60 + seconds

            timelines_times.append(timelines_time)
            timelines_sensors.append(timelines_sensor)
            timelines_infos.append(timelines_info)

    dtype = [('Time', int), ('adj_Time', int), ('Sensor', int), ('Information', 'S100')]

    timeline_data_np = np.array(list(zip(timelines_times, timelines_times, timelines_sensors, timelines_infos)), dtype=dtype)
            
    extreme_values_times = find_extreme_values(adj_qcmd_data, timeline_data_np)
    
    timeline_data_np["adj_Time"] = pd.Series(extreme_values_times)
    
    # Convert to pandas DataFrame
    dtype = ['Time', 'adj_Time', 'Sensor', 'Information']
    adj_timeline_data_pd = pd.DataFrame(timeline_data_np, columns=dtype)

    adj_timeline_data_pd['Information'] = adj_timeline_data_pd['Information'].apply(lambda x: x.decode('utf-8'))

    return adj_timeline_data_pd

    


In [10]:
def identify_similar_files(directory):
    files = os.listdir(directory)
    similar_files = [file for file in files if "rawdata.txt" in file.lower()]
    return similar_files

def main():
    rawdata_files = identify_similar_files(path)

    if not rawdata_files:
        print(f"There are no files containing 'rawdata.txt' in the folder {path}")
        return
    
    for i, file in enumerate(rawdata_files, start=1):
        print(f"{i}: {file}")
        file_path = os.path.join(path, file)

        persistentID, sensors, adj_qcmd_data = process_rawdata(file_path)
        
        if not persistentID:
            print("Error processing rawdata. Exiting.")
            return

        # Read timeline data using persistentID and path
        timeline_data = read_timeline_data(path, persistentID, adj_qcmd_data)

        if timeline_data is None:
            print("Error reading timeline data. Exiting.")
            return

        # Save sensor-specific adj_qcmd_data and adjusted timeline
        print("Data saved as:")
        
        baseID = persistentID.replace("_QCMD", "")
        csv_filename = os.path.join(path, "Data_" + baseID)
        adj_qcmd_data.to_csv(csv_filename + "_QCMD_adj_data.csv", index=True)
        print(csv_filename + "QCMD_adj_data.csv")

        # Loop through the sensors
        for sensor in sensors:
            # Save sensor-specific adjusted QCM-D data as a CSV file
            sensor_qcmd_data = adj_qcmd_data[adj_qcmd_data["Sensor"] == sensor].copy()
            del sensor_qcmd_data["Sensor"]
            adj_qcmd_data_name = f"{csv_filename}_S{sensor}_QCMD_adj_qcmd_data.csv"
            sensor_qcmd_data.to_csv(adj_qcmd_data_name, index=True)
            print(adj_qcmd_data_name)

            # Save sensor-specific adjusted timeline as a CSV file
            adj_timeline_filename = f"{csv_filename}_S{sensor}_QCMD_adj_timeline.csv"
            adj_sensor_timeline = timeline_data[timeline_data["Sensor"] == sensor].copy()
            del adj_sensor_timeline["Sensor"]
            adj_sensor_timeline.to_csv(adj_timeline_filename, index=True)
            print(adj_timeline_filename)


        print("")



if __name__ == "__main__":
    main()


1: Data_CBP_LEAF_5303643_20250514_QCMD_rawdata.txt
Starting to process data with the persistent ID: CBP_LEAF_5303643_20250514_QCMD
In this dataset, the following overtones were measured:
{3, 5, 7, 9, 11, 13}
on the sensor positions
{1, 2, 3, 4}
Transformed to normalized long format
Timeline file read:
Data_CBP_LEAF_5303643_20250514_QCMD_timeline.txt
Data saved as:
/Users/hboehm/Seafile/LEAF/QCMD/QCMD_data/Inbox/Data_CBP_LEAF_5303643_20250514QCMD_adj_data.csv
/Users/hboehm/Seafile/LEAF/QCMD/QCMD_data/Inbox/Data_CBP_LEAF_5303643_20250514_S1_QCMD_adj_qcmd_data.csv
/Users/hboehm/Seafile/LEAF/QCMD/QCMD_data/Inbox/Data_CBP_LEAF_5303643_20250514_S1_QCMD_adj_timeline.csv
/Users/hboehm/Seafile/LEAF/QCMD/QCMD_data/Inbox/Data_CBP_LEAF_5303643_20250514_S2_QCMD_adj_qcmd_data.csv
/Users/hboehm/Seafile/LEAF/QCMD/QCMD_data/Inbox/Data_CBP_LEAF_5303643_20250514_S2_QCMD_adj_timeline.csv
/Users/hboehm/Seafile/LEAF/QCMD/QCMD_data/Inbox/Data_CBP_LEAF_5303643_20250514_S3_QCMD_adj_qcmd_data.csv
/Users/hboehm/